In [2]:
import os
import h5py
import pandas as pd
import numpy as np
import wonambi as wpi
 
base_dir=r"C:\Users\byan\chb\chb-mit"
def get_content(fn,base_dir=base_dir):
  with open(os.path.join(*[base_dir,fn,fn+"-summary.txt"]), encoding='UTF-8') as f:
      # read all the document into a list of strings (each line a new string)
      content = f.readlines()

  return content


C:\Users\byan\AppData\Local\Temp\ipykernel_10900\3768267211.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import re
part_info_dict = {}

def info_dict(content):
  
  line_nos=len(content)
  line_no=1

  channels = []
  file_name = []
  file_info_dict={}

  for line in content:

    # if there is Channel in the line...
    if re.findall('Channel \d+', line):
      # split the line into channel number and channel reference
      channel = line.split(': ')
      # get the channel reference and remove any new lines
      channel = channel[-1].replace("\n", "")
      # put into the channel list
      channels.append(channel)

    # if the line is the file name
    elif re.findall('File Name', line):
      # if there is already a file_name
      if file_name:
        # flush the current file info to it
        part_info_dict[file_name] = file_info_dict

      # get the file name
      file_name = re.findall('\w+\d+_\d+|\w+\d+\w+_\d+', line)[0]

      file_info_dict = {}
      # put the channel list in the file info dict and remove duplicates
      file_info_dict['Channels'] = list(set(channels))
      # reset the rest of the options
      file_info_dict['Start Time'] = ''
      file_info_dict['End Time'] = ''
      file_info_dict['Seizures Window'] = []

    # if the line is about the file start time
    elif re.findall('File Start Time', line):
      # get the start time
      file_info_dict['Start Time'] = re.findall('\d+:\d+:\d+', line)[0]

    # if the line is about the file end time
    elif re.findall('File End Time', line):
      # get the start time
      file_info_dict['End Time'] = re.findall('\d+:\d+:\d+', line)[0]

    elif re.findall('Seizure Start Time|Seizure End Time|Seizure \d+ Start Time|Seizure \d+ End Time', line):
      file_info_dict['Seizures Window'].append(int(re.findall('\d+', line)[-1]))

    # if last line in the list...
    if line_no == line_nos:
      # flush the file info to it
      part_info_dict[file_name] = file_info_dict

    line_no+=1
    
        
for part_code in os.listdir(r"C:\Users\byan\chb\chb-mit"):
  if os.path.isdir(os.path.join(base_dir, part_code)):
    content = get_content(part_code)
    info_dict(content)

In [4]:
files_df=pd.DataFrame(part_info_dict).T
files_df = files_df[~files_df.index.to_series().str.contains('chb24')]
files_df.head()

,Channels,Start Time,End Time,Seizures Window
chb01_01,"[FP1-F7, C3-P3, FT9-FT10, P8-O2, FT10-T8, FP1-...",11:42:54,12:42:54,[]
chb01_02,"[FP1-F7, C3-P3, FT9-FT10, P8-O2, FT10-T8, FP1-...",12:42:57,13:42:57,[]
chb01_03,"[FP1-F7, C3-P3, FT9-FT10, P8-O2, FT10-T8, FP1-...",13:43:04,14:43:04,"[2996, 3036]"
chb01_04,"[FP1-F7, C3-P3, FT9-FT10, P8-O2, FT10-T8, FP1-...",14:43:12,15:43:12,"[1467, 1494]"
chb01_05,"[FP1-F7, C3-P3, FT9-FT10, P8-O2, FT10-T8, FP1-...",15:43:19,16:43:19,[]


In [5]:
files_df.Channels=files_df.Channels.map(lambda x: set(x))
channels_keep=set.intersection(*files_df.Channels)  # find common channels among all records
files_df.drop(labels=['Channels'],axis=1,inplace=True)

In [6]:
seizure_files=files_df.loc[files_df['Seizures Window'].map(lambda x: len(x)>=2) ]  
no_seizure_files= files_df.loc[files_df['Seizures Window'].map(lambda x: len(x)<2)]  


In [7]:
def start_end_pair(array):
  """
  in  > [1,2,3,4,5,6]          
  out > [(1,2),(3,4),(5,6)]    start,end pairs for each recording period  
  """
  pairs=[]
  for i in range(0,len(array),2):
    pairs.append((array[i],array[i+1]))
  return pairs

def find_total_time_sec(start,end):
  """
  s : pd.Series object
  """
  #s=s.map(lambda x:pd.to_datetime(x, format='%H:%M:%S') )
  start=list( start.map(lambda x: (str(x).strip('][').split(":")) ).map(lambda x: [int(i) for i in x] ) )
  end=list(end.map(lambda x: (str(x).strip('][').split(":")) ).map(lambda x: [int(i) for i in x] ) ) 
  diff=pd.Series()
  for i,(s,e) in enumerate(zip(start,end)):
    diff.loc[i]=(e[0]-s[0])*60*60 + (e[1]-s[1])*60 +(e[2]-s[2])
  return diff.values

In [8]:

no_seizure_files["total_sec"]= find_total_time_sec(no_seizure_files['Start Time'],no_seizure_files['End Time'])


C:\Users\byan\AppData\Local\Temp\ipykernel_10900\239066888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_seizure_files["total_sec"]= find_total_time_sec(no_seizure_files['Start Time'],no_seizure_files['End Time'])


In [9]:
no_seizure_files.drop(["Start Time" ,	"End Time" ,	"Seizures Window"],axis=1,inplace=True)

C:\Users\byan\AppData\Local\Temp\ipykernel_10900\1230261561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_seizure_files.drop(["Start Time" ,	"End Time" ,	"Seizures Window"],axis=1,inplace=True)


In [10]:
seizure_files.loc[:,"Seizures Window"]=seizure_files["Seizures Window"].map(start_end_pair )
seizure_files=seizure_files.explode("Seizures Window")

In [11]:
from sklearn.model_selection import train_test_split
no_seizure_files_train,no_seizure_files_test=train_test_split( no_seizure_files ,random_state=0 )
seizure_files_train,seizure_files_test=train_test_split( seizure_files ,random_state=0)


In [12]:
def no_seizure_window(total_sec,window_size=15,samples_per_record=20 ):
  starts=np.random.randint(0,high= total_sec - window_size , size=samples_per_record)
  return list(map(lambda x : (x,x+window_size),starts ))

def seizure_window(start_end,val=False,window_size=15):
  start,end=start_end
  l=[]
  i=start
  while i < end-1:
    l.append((i,i+window_size))
    if val:
      i+=window_size
    else:
      i+=1
  return l
seizure_files_train['Seizures Window']=seizure_files_train['Seizures Window'].map(lambda x : seizure_window(x) )
seizure_files_train=seizure_files_train.explode("Seizures Window")

seizure_files_test['Seizures Window']=seizure_files_test['Seizures Window'].map(lambda x : seizure_window(x,val=True) )
seizure_files_test=seizure_files_test.explode("Seizures Window")

no_seizure_files_train["Seizures Window"]=no_seizure_files_train.total_sec.map(no_seizure_window)
no_seizure_files_train=no_seizure_files_train.explode("Seizures Window")

no_seizure_files_test["Seizures Window"]=no_seizure_files_test.total_sec.map(lambda x: no_seizure_window(x,samples_per_record=10))
no_seizure_files_test=no_seizure_files_test.explode("Seizures Window")



In [13]:
no_seizure_files_train.drop(['total_sec'],axis=1,inplace=True)
no_seizure_files_test.drop(['total_sec'],axis=1,inplace=True)


In [14]:
seizure_files_train['label']=1
seizure_files_test['label']=1

no_seizure_files_train["label"]=0
no_seizure_files_test["label"]=0


training_df=pd.concat([seizure_files_train,no_seizure_files_train],axis=0)
testing_df=pd.concat([seizure_files_test,no_seizure_files_test],axis=0)

training_df=training_df.sample(frac= 1) 
testing_df=testing_df.sample(frac=1)

In [15]:
training_df.head()

,Start Time,End Time,Seizures Window,label
chb15_61,NaN,NaN,"(3226, 3241)",0
chb05_16,08:22:35,9:22:35,"(2402, 2417)",1
chb08_05,15:29:14,16:29:14,"(3027, 3042)",1
chb06_15,NaN,NaN,"(6403, 6418)",0
chb03_06,NaN,NaN,"(413, 428)",0


In [16]:
training_df["Seizures Window"]=training_df["Seizures Window"].map(lambda x : x [0])
testing_df["Seizures Window"]=testing_df["Seizures Window"].map(lambda x : x [0])

training_df.to_csv("data/training.csv") 
testing_df.to_csv("data/testing_df.csv") 


In [17]:
training_df.head()

,Start Time,End Time,Seizures Window,label
chb15_61,NaN,NaN,3226,0
chb05_16,08:22:35,9:22:35,2402,1
chb08_05,15:29:14,16:29:14,3027,1
chb06_15,NaN,NaN,6403,0
chb03_06,NaN,NaN,413,0


In [18]:

def read_edf(file_name,start_t=0,window=15,s_rate=256):
  try:
      global channels_keep
      file_name=os.path.join(base_dir,file_name[:5],str(file_name)+".edf")
      f=wpi.ioeeg.edf.Edf(file_name)
      sig_labels=f.return_hdr()[3]
      try:
        idx= list(map(lambda x: sig_labels.index(x), channels_keep )) 
      except Exception as ex:
        channels_keep_first=list(map(lambda x: x.split('-')[0] , channels_keep))
        sig_labels=list(map(lambda x: x.split('-')[0] if len(x)>2 else x[:2] ,sig_labels))
        idx= list(map(lambda x: sig_labels.index(x), channels_keep_first ))
        print(e)
        print(file_name)
        return None
      sigbufs=f.return_dat(idx,start_t*s_rate,start_t*s_rate+window*s_rate)
  except OSError as e:
        print('cannot open', file_name)
        print(e)
        return None
  
  except Exception as e:
    print(e)
    print(file_name)

    
    return None


  return sigbufs.T

In [ ]:
with h5py.File("data/train.hdf5", "w") as f :
  y=f.create_dataset("y", (training_df.shape[0],),dtype='i')
  y[:]=training_df.label.values
  X = f.create_dataset("X", (training_df.shape[0],15*256, 18), dtype='f')


  for i,(index,row) in enumerate(training_df.iterrows()):
    raw=read_edf(index,row['Seizures Window'])
    if raw is not None:
      X[i]=raw
      print(i)
    else:
      print("error reading {}".format(index))


In [ ]:
with h5py.File("data/test.hdf5", "w") as f :
  y=f.create_dataset("y", (testing_df.shape[0],),dtype='i')
  y[:]=testing_df.label.values
  X = f.create_dataset("X", (training_df.shape[0],15*256, 18), dtype='f')


  for i,(index,row) in enumerate(testing_df.iterrows()):
    raw=read_edf(index,row['Seizures Window'])
    if raw is not None:
      X[i]=raw
      print(i)
    else:
      print("error reading {}".format(index))


In [26]:
training_df.shape

(15794, 4)

In [27]:
testing_df.shape

(1566, 4)